# Use duckdb python api

In this notebook, we will use duckdb python api to explore some functionality of the duck db. 

In [4]:
import pandas as pd
import duckdb

In [5]:
root_dir = "/home/pengfei/data_set/demo_chu"
csv_file_path = f"{root_dir}/pathologies.csv"
parquet_file_path = f"{root_dir}/pathologies.parquet"

In [6]:
! ls -lah /home/pengfei/data_set/demo_chu

total 683M
drwxrwxr-x  3 pengfei pengfei 4.0K May 22 13:55 .
drwxrwxr-x 13 pengfei pengfei 4.0K May 22 12:03 ..
drwxrwxr-x  2 pengfei pengfei 4.0K May 22 14:49 demo_base
-rw-rw-r--  1 pengfei pengfei 657M May 16 15:54 pathologies.csv.bkp
-rw-rw-r--  1 pengfei pengfei  26M May 17 09:03 pathologies.parquet


## Create a duckdb instance

As we mentioned in the introduction, duckdb has two modes:
- in-memory: duckdb.connect()
- on-disk: duckdb.connect("path/to/file")

In [7]:
## Create an in-memory duckdb instance 
# conn = duckdb.connect()

# create an on disk instance, you can also activate the read only option
conn = duckdb.connect("/home/pengfei/data_set/demo_chu/demo_base/py_mydb.db")

## 1. Compare the data loading speed 

In this section, we compare the data loading speed between duck db and pandas

### 1.1 Compare the reading speed of  csv


In [9]:
%%time
# read csv
csv_query = f"""create or replace view patho_csv as select * from read_csv('{csv_file_path}', header=true, delim = ';'); select count(*) from patho_csv"""

# this will return a pandas dataframe
csv_row_count = conn.execute(csv_query).df()

print(type(csv_row_count))
csv_row_count.head(5)

<class 'pandas.core.frame.DataFrame'>
CPU times: user 2.01 s, sys: 883 ms, total: 2.89 s
Wall time: 1.37 s


,count_star()
0,4057201


> we can notice the execution time of count row number action takes about 1.5 seconds

Now let's try to read the csv with pandas


In [35]:
%%time
csv_pdf = pd.read_csv(csv_file_path, sep=";")

print(f"row count: {len(csv_pdf)}")

<timed exec>:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.


row count: 4057201
CPU times: user 8.86 s, sys: 1.31 s, total: 10.2 s
Wall time: 10.2 s


In [7]:
row_number, col_number = csv_pdf.shape
print(f"The data set contains {row_number} rows and {col_number} columns")

The data set contains 4057201 rows and 16 columns


> we can notice the execution time for pandas is about 11 seconds. so we gain about 7 times the execution time

### 1.2 Compare the reading speed of parquet


In [11]:
%%time
# read parquet with duck db
parquet_query = f"""create or replace view patho_parquet as select * from read_parquet('{parquet_file_path}'); select count(*) from patho_parquet"""

# this will return a pandas dataframe
parquet_row_count = conn.execute(parquet_query).df()

parquet_row_count.head(5)

CPU times: user 2.65 ms, sys: 3.91 ms, total: 6.56 ms
Wall time: 14.7 ms


,count_star()
0,4057201


> With parquet and duckdb, we can reduce the data loading time to 4 ms

In [12]:
%%time
# read parquet with pandas

parquet_pdf = pd.read_parquet(parquet_file_path, engine='pyarrow')
print(f"row count: {len(parquet_pdf)}")

row count: 4057201
CPU times: user 1.92 s, sys: 843 ms, total: 2.77 s
Wall time: 1.48 s


> pandas can't read parquet natively, it requires pyarrow or fastparquet, here we use the pyarrow package. But still duckdb wins on the count operation

## 2. Compare the different action times

### 2.1 Get the table schema

In [13]:
%%time
table_name = "patho_csv"
query2 = f"Describe {table_name}"
schema = conn.execute(query2).df()
schema.head(15)

CPU times: user 0 ns, sys: 1.73 ms, total: 1.73 ms
Wall time: 1.37 ms


,column_name,column_type,null,key,default,extra
0,annee,BIGINT,YES,None,None,None
1,patho_niv1,VARCHAR,YES,None,None,None
2,patho_niv2,VARCHAR,YES,None,None,None
3,patho_niv3,VARCHAR,YES,None,None,None
4,top,VARCHAR,YES,None,None,None
5,cla_age_5,VARCHAR,YES,None,None,None
6,sexe,BIGINT,YES,None,None,None
7,region,VARCHAR,YES,None,None,None
8,dept,VARCHAR,YES,None,None,None
9,ntop,BIGINT,YES,None,None,None


In [14]:
%%time
print(csv_pdf.dtypes)

annee                 float64
patho_niv1             object
patho_niv2             object
patho_niv3             object
top                    object
cla_age_5              object
sexe                  float64
region                  int64
dept                   object
ntop                  float64
npop                  float64
prev                  float64
niveau_prioritaire     object
libelle_classe_age     object
libelle_sexe           object
tri                   float64
dtype: object
CPU times: user 958 µs, sys: 0 ns, total: 958 µs
Wall time: 950 µs


> This time the winner is pyarrow/pandas

### Test Filter query speed


In [24]:
%%time
parquet_filter_query = f"select annee, sexe, dept from patho_parquet where (sexe = 9.0) AND (dept = '75')"

parquet_filter_res = conn.execute(parquet_filter_query).df()

parquet_filter_res.head()

CPU times: user 139 ms, sys: 15.9 ms, total: 155 ms
Wall time: 150 ms


,annee,sexe,dept
0,2017.0,9.0,75
1,2017.0,9.0,75
2,2017.0,9.0,75
3,2017.0,9.0,75
4,2017.0,9.0,75


In [34]:
%%time
csv_filter_query = f"select annee, sexe, dept from patho_csv where (sexe = 9.0) AND (dept = '75')"

csv_filter_res = conn.execute(csv_filter_query).df()

csv_filter_res.head()

CPU times: user 3.47 s, sys: 594 ms, total: 4.06 s
Wall time: 2.12 s


,annee,sexe,dept
0,2017,9,75
1,2017,9,75
2,2017,9,75
3,2017,9,75
4,2017,9,75


In [32]:
%%time

parquet_filter_pdf = parquet_pdf[(parquet_pdf['sexe']==9.0) & (parquet_pdf['dept']=='75')][['annee','sexe','dept']]
parquet_filter_pdf.head()

CPU times: user 330 ms, sys: 4.47 ms, total: 334 ms
Wall time: 348 ms


,annee,sexe,dept
110,2017.0,9.0,75
364,2017.0,9.0,75
733,2017.0,9.0,75
1087,2017.0,9.0,75
1351,2017.0,9.0,75


In [36]:
%%time

csv_filter_pdf = csv_pdf[(csv_pdf['sexe']==9.0) & (csv_pdf['dept']=='75')][['annee','sexe','dept']]
csv_filter_pdf.head()

CPU times: user 330 ms, sys: 6.45 ms, total: 337 ms
Wall time: 341 ms


,annee,sexe,dept
110,2017.0,9.0,75
364,2017.0,9.0,75
733,2017.0,9.0,75
1087,2017.0,9.0,75
1351,2017.0,9.0,75


> We can notice that pandas wins on csv dataframe, but duckdb wins on the parquet dataframe. That's because pandas loads all data into the memory too. DuckDB does lazy loading on the persist tables.

## Test group by query speed

In [37]:
%%time
parquet_gb_query = f"SELECT dept, sexe, annee, patho_niv1, SUM(ntop), AVG(npop) FROM patho_parquet GROUP BY ALL;"

parquet_gb_res = conn.execute(parquet_gb_query).df()

parquet_gb_res.head()

CPU times: user 1.39 s, sys: 101 ms, total: 1.49 s
Wall time: 1.15 s


,dept,sexe,annee,patho_niv1,sum(ntop),avg(npop)
0,42,9.0,2017.0,"Pas de pathologies repérées, traitements, mate...",774360.0,70993.333333
1,971,1.0,2017.0,"Pas de pathologies repérées, traitements, mate...",204060.0,16185.238095
2,18,1.0,2017.0,"Pas de pathologies repérées, traitements, mate...",147580.0,13358.571429
3,55,1.0,2017.0,"Pas de pathologies repérées, traitements, mate...",89910.0,8092.857143
4,68,1.0,2017.0,"Pas de pathologies repérées, traitements, mate...",408810.0,33312.857143
